# Expresion
Datos de expresión extraidos de:
https://ccb-web.cs.uni-saarland.de/tissueatlas2/downloads

In [1]:
import pandas as pd 
import seaborn as sns
import multiprocessing as mp
from tqdm import tqdm
import ast
import matplotlib.pyplot as plt

In [2]:
dft = pd.read_csv('../Data/Contribution_miRNA_time_pathwayGO_BP.tsv', sep = '\t')
dfv = pd.read_csv('../Data/Contribution_miRNA_variant_pathwayGO_BP.tsv', sep = '\t')
dfv.head()

,function,p_value,miRNA,Variant,DuplexEnergy,Jaccard,Region
0,circadian behavior,0.034054,hsa-miR-638,Global,-34.5,0.84,5'UTR
1,glycerophospholipid biosynthetic process,0.034054,hsa-miR-638,Global,-34.5,0.84,5'UTR
2,negative regulation of G1/S transition of mito...,0.034054,hsa-miR-638,Global,-34.5,0.84,5'UTR
3,negative regulation of cell cycle G1/S phase t...,0.034054,hsa-miR-638,Global,-34.5,0.84,5'UTR
4,negative regulation of epithelial to mesenchym...,0.034054,hsa-miR-638,Global,-34.5,0.84,5'UTR


In [3]:
# estan en el primero, pero no en el segundo 
len(set(dft[dft['Variant']=='(2010-12-05,2020-03-01]']['miRNA']) - set(dft[dft['Variant']=='Global']['miRNA']))

20

In [4]:
df_expression = pd.read_csv('../Data/hsa_snc_expression.csv')
df_expression.head()

,source,norm,body,organ,tissue,type,acc,expression,species
0,ngs,rpmm,3,adipocyte,adipocyte,trna,tRNA-Ala-AGC-1-1,88.905542,hsa
1,ngs,rpmm,3,adipocyte,adipocyte,trna,tRNA-Ala-AGC-10-1,0.000000,hsa
2,ngs,rpmm,3,adipocyte,adipocyte,trna,tRNA-Ala-AGC-10-2,0.000000,hsa
3,ngs,rpmm,3,adipocyte,adipocyte,trna,tRNA-Ala-AGC-11-1,47.789356,hsa
4,ngs,rpmm,3,adipocyte,adipocyte,trna,tRNA-Ala-AGC-12-1,0.000000,hsa


In [5]:
tissues_ACE2 = ['brain', 'esophagus', 'lung', 'heart', 'kidney', 'stomach', 'gallbladder', 'liver', 'pancreas', 'bowel', 'vein', 'artery']
df_expression_sars = df_expression[df_expression['organ'].isin(tissues_ACE2)]
df_expression_sars = df_expression_sars[df_expression_sars['expression'] > 0]
df_expression_sars = df_expression_sars[df_expression_sars['type'] == 'mirna']
df_expression_sars

,source,norm,body,organ,tissue,type,acc,expression,species
171936,ngs,rpmm,1,kidney,kidney,mirna,hsa-let-7a-2-3p,0.500669,hsa
171937,ngs,rpmm,1,kidney,kidney,mirna,hsa-let-7a-3p,95.600018,hsa
171938,ngs,rpmm,1,kidney,kidney,mirna,hsa-let-7a-5p,2403.045671,hsa
171939,ngs,rpmm,1,kidney,kidney,mirna,hsa-let-7b-3p,13.907480,hsa
171940,ngs,rpmm,1,kidney,kidney,mirna,hsa-let-7b-5p,9122.861906,hsa
...,...,...,...,...,...,...,...,...,...
16411643,ngs,rpmm,3,liver,liver,mirna,hsa-miR-9986,0.251370,hsa
16411644,ngs,rpmm,3,liver,liver,mirna,hsa-miR-99a-3p,117.724918,hsa
16411645,ngs,rpmm,3,liver,liver,mirna,hsa-miR-99a-5p,3356.039952,hsa
16411646,ngs,rpmm,3,liver,liver,mirna,hsa-miR-99b-3p,6.591478,hsa


In [6]:
def extract_expression(miRNA):
    if miRNA in list(df_expression_sars['acc']):
        keys =  df_expression_sars[df_expression_sars['acc'] == miRNA]['organ']
        values = df_expression_sars[df_expression_sars['acc'] == miRNA]['expression']
        dictionary = dict(zip(keys, values))
        return dictionary
    else:
        return 0

In [7]:
# Crea un pool de procesos
pool = mp.Pool(mp.cpu_count())
pbar = tqdm(total=len(dft['miRNA']))
def update(*a):
    pbar.update()
results = []
for miRNA in dft['miRNA']:
    result = pool.apply_async(extract_expression, args=(miRNA,), callback=update)
    results.append(result)
pool.close()
pool.join()
dft['Expression'] = [result.get() for result in results]
pbar.close()

print("Done")

100%|██████████| 135614/135614 [11:58<00:00, 188.65it/s]

Done


In [8]:
dft.head()

,function,p_value,miRNA,Variant,DuplexEnergy,Jaccard,Region,Expression
0,circadian behavior,0.034054,hsa-miR-638,"(2022-11-01,2023-01-01]",-34.5,0.84,5'UTR,"{'bowel': 0.0834956488747458, 'brain': 0.06946..."
1,glycerophospholipid biosynthetic process,0.034054,hsa-miR-638,"(2022-11-01,2023-01-01]",-34.5,0.84,5'UTR,"{'bowel': 0.0834956488747458, 'brain': 0.06946..."
2,negative regulation of G1/S transition of mito...,0.034054,hsa-miR-638,"(2022-11-01,2023-01-01]",-34.5,0.84,5'UTR,"{'bowel': 0.0834956488747458, 'brain': 0.06946..."
3,negative regulation of cell cycle G1/S phase t...,0.034054,hsa-miR-638,"(2022-11-01,2023-01-01]",-34.5,0.84,5'UTR,"{'bowel': 0.0834956488747458, 'brain': 0.06946..."
4,negative regulation of epithelial to mesenchym...,0.034054,hsa-miR-638,"(2022-11-01,2023-01-01]",-34.5,0.84,5'UTR,"{'bowel': 0.0834956488747458, 'brain': 0.06946..."


In [11]:
# Crea un pool de procesos
pool = mp.Pool(mp.cpu_count())
pbar = tqdm(total=len(dfv['miRNA']))
def update(*a):
    pbar.update()
results = []
for miRNA in dfv['miRNA']:
    result = pool.apply_async(extract_expression, args=(miRNA,), callback=update)
    results.append(result)
pool.close()
pool.join()
dfv['Expression'] = [result.get() for result in results]
pbar.close()

print("Done")

100%|██████████| 533316/533316 [47:11<00:00, 188.33it/s]

Done


In [12]:
dfv.head()

,function,p_value,miRNA,Variant,DuplexEnergy,Jaccard,Region,Expression
0,circadian behavior,0.034054,hsa-miR-638,Global,-34.5,0.84,5'UTR,"{'bowel': 0.0834956488747458, 'brain': 0.06946..."
1,glycerophospholipid biosynthetic process,0.034054,hsa-miR-638,Global,-34.5,0.84,5'UTR,"{'bowel': 0.0834956488747458, 'brain': 0.06946..."
2,negative regulation of G1/S transition of mito...,0.034054,hsa-miR-638,Global,-34.5,0.84,5'UTR,"{'bowel': 0.0834956488747458, 'brain': 0.06946..."
3,negative regulation of cell cycle G1/S phase t...,0.034054,hsa-miR-638,Global,-34.5,0.84,5'UTR,"{'bowel': 0.0834956488747458, 'brain': 0.06946..."
4,negative regulation of epithelial to mesenchym...,0.034054,hsa-miR-638,Global,-34.5,0.84,5'UTR,"{'bowel': 0.0834956488747458, 'brain': 0.06946..."


In [9]:
dft.to_csv('../Data/miRNA_expression_pathways_time.tsv', index = False, sep = '\t')
dfv.to_csv('../Data/miRNA_expression_pathways_variant.tsv', index = False, sep = '\t')

# Inspection

In [10]:
dft = pd.read_csv('../Data/miRNA_expression_pathways_time.tsv', sep = '\t')
dfv = pd.read_csv('../Data/miRNA_expression_pathways_variant.tsv', sep = '\t')
len(dfv)

533316

In [11]:
dft['Expression'] = dft['Expression'].apply(ast.literal_eval)
expanded_dft = dft['Expression'].apply(pd.Series)
dft = pd.concat([dft, expanded_dft], axis=1)


dfv['Expression'] = dfv['Expression'].apply(ast.literal_eval)
expanded_dfv = dfv['Expression'].apply(pd.Series)
dfv = pd.concat([dfv, expanded_dfv], axis=1)
dfv.head()

In [12]:
dft.to_csv('../Data/Expanded_miRNAPredicted_SinEstructuras_time.tsv', sep = '\t', index = False)
dfv.to_csv('../Data/Expanded_miRNAPredicted_SinEstructuras_variant.tsv', sep = '\t', insex = False)

# Filtro

In [13]:
dft = pd.read_csv('../Data/Expanded_miRNAPredicted_SinEstructuras_time.tsv', sep = '\t')
dfv = pd.read_csv('../Data/Expanded_miRNAPredicted_SinEstructuras_variant.tsv', sep = '\t')

In [15]:
dfft = dft[dft['lung']>10]
dfft

,function,p_value,miRNA,Variant,DuplexEnergy,Jaccard,Region,Expression,bowel,brain,...,lung,esophagus,kidney,artery,heart,vein,pancreas,stomach,liver,0
897,cellular response to hormone stimulus,0.045992,hsa-miR-486-5p,"(2021-05-01,2022-01-01]",-26.0,0.837209,5'UTR,"{'kidney': 24.10987895245845, 'brain': 10.3713...",7.042559,10.371314,...,207.324974,2.556688,24.109879,36.132875,17.769939,809.631661,20.861899,4.212078,11.088207,NaN
898,cellular response to insulin stimulus,0.007175,hsa-miR-486-5p,"(2021-05-01,2022-01-01]",-26.0,0.837209,5'UTR,"{'kidney': 24.10987895245845, 'brain': 10.3713...",7.042559,10.371314,...,207.324974,2.556688,24.109879,36.132875,17.769939,809.631661,20.861899,4.212078,11.088207,NaN
899,cellular response to insulin-like growth facto...,0.018656,hsa-miR-486-5p,"(2021-05-01,2022-01-01]",-26.0,0.837209,5'UTR,"{'kidney': 24.10987895245845, 'brain': 10.3713...",7.042559,10.371314,...,207.324974,2.556688,24.109879,36.132875,17.769939,809.631661,20.861899,4.212078,11.088207,NaN
900,cellular response to organonitrogen compound,0.037773,hsa-miR-486-5p,"(2021-05-01,2022-01-01]",-26.0,0.837209,5'UTR,"{'kidney': 24.10987895245845, 'brain': 10.3713...",7.042559,10.371314,...,207.324974,2.556688,24.109879,36.132875,17.769939,809.631661,20.861899,4.212078,11.088207,NaN
901,cellular response to peptide,0.007394,hsa-miR-486-5p,"(2021-05-01,2022-01-01]",-26.0,0.837209,5'UTR,"{'kidney': 24.10987895245845, 'brain': 10.3713...",7.042559,10.371314,...,207.324974,2.556688,24.109879,36.132875,17.769939,809.631661,20.861899,4.212078,11.088207,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135609,regulation of superoxide anion generation,0.018746,hsa-miR-486-3p,"(2023-07-01,2023-09-06]",-10.0,0.896552,5'UTR,"{'kidney': 8.692315785505969, 'brain': 6.45561...",1.269577,6.455614,...,12.067047,2.173184,8.692316,18.534121,12.700229,281.132254,8.098955,1.753135,4.901714,NaN
135610,regulation of superoxide metabolic process,0.021742,hsa-miR-486-3p,"(2023-07-01,2023-09-06]",-10.0,0.896552,5'UTR,"{'kidney': 8.692315785505969, 'brain': 6.45561...",1.269577,6.455614,...,12.067047,2.173184,8.692316,18.534121,12.700229,281.132254,8.098955,1.753135,4.901714,NaN
135611,stimulatory C-type lectin receptor signaling p...,0.049946,hsa-miR-486-3p,"(2023-07-01,2023-09-06]",-10.0,0.896552,5'UTR,"{'kidney': 8.692315785505969, 'brain': 6.45561...",1.269577,6.455614,...,12.067047,2.173184,8.692316,18.534121,12.700229,281.132254,8.098955,1.753135,4.901714,NaN
135612,superoxide anion generation,0.021742,hsa-miR-486-3p,"(2023-07-01,2023-09-06]",-10.0,0.896552,5'UTR,"{'kidney': 8.692315785505969, 'brain': 6.45561...",1.269577,6.455614,...,12.067047,2.173184,8.692316,18.534121,12.700229,281.132254,8.098955,1.753135,4.901714,NaN


In [16]:
dffv = dfv[dfv['lung']>10]
dffv

,Unnamed: 0,function,p_value,miRNA,Variant,DuplexEnergy,Jaccard,Region,Expression,bowel,...,lung,esophagus,kidney,artery,heart,vein,pancreas,stomach,liver,0
2669,2669,cellular response to hormone stimulus,0.045992,hsa-miR-486-5p,20J,-26.0,0.837209,5'UTR,"{'kidney': 24.10987895245845, 'brain': 10.3713...",7.042559,...,207.324974,2.556688,24.109879,36.132875,17.769939,809.631661,20.861899,4.212078,11.088207,NaN
2670,2670,cellular response to insulin stimulus,0.007175,hsa-miR-486-5p,20J,-26.0,0.837209,5'UTR,"{'kidney': 24.10987895245845, 'brain': 10.3713...",7.042559,...,207.324974,2.556688,24.109879,36.132875,17.769939,809.631661,20.861899,4.212078,11.088207,NaN
2671,2671,cellular response to insulin-like growth facto...,0.018656,hsa-miR-486-5p,20J,-26.0,0.837209,5'UTR,"{'kidney': 24.10987895245845, 'brain': 10.3713...",7.042559,...,207.324974,2.556688,24.109879,36.132875,17.769939,809.631661,20.861899,4.212078,11.088207,NaN
2672,2672,cellular response to organonitrogen compound,0.037773,hsa-miR-486-5p,20J,-26.0,0.837209,5'UTR,"{'kidney': 24.10987895245845, 'brain': 10.3713...",7.042559,...,207.324974,2.556688,24.109879,36.132875,17.769939,809.631661,20.861899,4.212078,11.088207,NaN
2673,2673,cellular response to peptide,0.007394,hsa-miR-486-5p,20J,-26.0,0.837209,5'UTR,"{'kidney': 24.10987895245845, 'brain': 10.3713...",7.042559,...,207.324974,2.556688,24.109879,36.132875,17.769939,809.631661,20.861899,4.212078,11.088207,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
533311,533311,tube development,0.037379,hsa-miR-140-5p,20G,-10.0,0.842105,3'UTR,"{'kidney': 881.5124650732382, 'brain': 961.172...",291.331975,...,245.662734,641.408980,881.512465,2948.137719,1664.491363,1612.359115,635.563109,930.846468,774.777988,NaN
533312,533312,tube formation,0.027079,hsa-miR-140-5p,20G,-10.0,0.842105,3'UTR,"{'kidney': 881.5124650732382, 'brain': 961.172...",291.331975,...,245.662734,641.408980,881.512465,2948.137719,1664.491363,1612.359115,635.563109,930.846468,774.777988,NaN
533313,533313,vascular smooth muscle cell differentiation,0.022198,hsa-miR-140-5p,20G,-10.0,0.842105,3'UTR,"{'kidney': 881.5124650732382, 'brain': 961.172...",291.331975,...,245.662734,641.408980,881.512465,2948.137719,1664.491363,1612.359115,635.563109,930.846468,774.777988,NaN
533314,533314,ventricular septum morphogenesis,0.041438,hsa-miR-140-5p,20G,-10.0,0.842105,3'UTR,"{'kidney': 881.5124650732382, 'brain': 961.172...",291.331975,...,245.662734,641.408980,881.512465,2948.137719,1664.491363,1612.359115,635.563109,930.846468,774.777988,NaN


In [17]:
dfft.to_csv('../Data/Expanded_miRNAPredicted_SinEstructuras_time_Filter.tsv', sep = '\t', index = False)
dffv.to_csv('../Data/Expanded_miRNAPredicted_SinEstructuras_variant_Filter.tsv', sep = '\t', index = False)

# Extract Set

In [57]:
dfft['Variant'].unique()

array(['Global', '(2021-01-01,2021-05-01]', '(2010-12-05,2020-03-01]',
       '(2022-01-01,2022-06-01]', '(2022-11-01,2023-01-01]',
       '(2023-07-01,2023-09-06]', '(2020-03-01,2020-09-01]',
       '(2023-01-01,2023-07-01]', '(2021-05-01,2022-01-01]',
       '(2020-09-01,2021-01-01]', '(2022-06-01,2022-11-01]'], dtype=object)

In [58]:
# estan en el primero, pero no en el segundo 
len(set(dft[dft['Variant']=='(2010-12-05,2020-03-01]']['miRNA']) - set(dft[dft['Variant']=='(2022-01-01,2022-06-01]']['miRNA']))

5

In [59]:
# estan en el primero, pero no en el segundo 
len(set(dft[dft['Variant']=='(2010-12-05,2020-03-01]']['miRNA']) - set(dft[dft['Variant']=='(2022-11-01,2023-01-01]']['miRNA']))

5

In [60]:
df_bd = pd.DataFrame()
for i in dfft['Variant'].unique():
    x = dfft[dfft['Variant']==i]['miRNA'].drop_duplicates().reset_index(drop=True)
    df_bd[i] = x
df_bd

,Global,"(2021-01-01,2021-05-01]","(2010-12-05,2020-03-01]","(2022-01-01,2022-06-01]","(2022-11-01,2023-01-01]","(2023-07-01,2023-09-06]","(2020-03-01,2020-09-01]","(2023-01-01,2023-07-01]","(2021-05-01,2022-01-01]","(2020-09-01,2021-01-01]","(2022-06-01,2022-11-01]"
0,hsa-miR-486-5p,hsa-miR-486-5p,hsa-miR-486-5p,hsa-miR-486-5p,hsa-miR-486-5p,hsa-miR-486-5p,hsa-miR-486-5p,hsa-miR-486-5p,hsa-miR-486-5p,hsa-miR-486-5p,hsa-miR-486-5p
1,hsa-miR-503-5p,hsa-miR-503-5p,hsa-miR-503-5p,hsa-miR-503-5p,hsa-miR-503-5p,hsa-miR-503-5p,hsa-miR-503-5p,hsa-miR-503-5p,hsa-miR-503-5p,hsa-miR-503-5p,hsa-miR-503-5p
2,hsa-miR-34a-5p,hsa-miR-34a-5p,hsa-miR-34a-5p,hsa-miR-34a-5p,hsa-miR-34a-5p,hsa-miR-34a-5p,hsa-miR-34a-5p,hsa-miR-34a-5p,hsa-miR-34a-5p,hsa-miR-34a-5p,hsa-miR-34a-5p
3,hsa-miR-381-3p,hsa-miR-381-3p,hsa-miR-381-3p,hsa-miR-381-3p,hsa-miR-381-3p,hsa-miR-381-3p,hsa-miR-381-3p,hsa-miR-381-3p,hsa-miR-381-3p,hsa-miR-381-3p,hsa-miR-381-3p
4,hsa-miR-196b-5p,hsa-miR-196b-5p,hsa-miR-196b-5p,hsa-miR-196b-5p,hsa-miR-196b-5p,hsa-miR-196b-5p,hsa-miR-196b-5p,hsa-miR-196b-5p,hsa-miR-196b-5p,hsa-miR-196b-5p,hsa-miR-196b-5p
5,hsa-miR-18b-5p,hsa-miR-18b-5p,hsa-miR-18b-5p,hsa-miR-18b-5p,hsa-miR-18b-5p,hsa-miR-18b-5p,hsa-miR-18b-5p,hsa-miR-18b-5p,hsa-miR-18b-5p,hsa-miR-18b-5p,hsa-miR-18b-5p
6,hsa-let-7b-5p,hsa-let-7b-5p,hsa-let-7b-5p,hsa-let-7b-5p,hsa-let-7b-5p,hsa-let-7b-5p,hsa-let-7b-5p,hsa-let-7b-5p,hsa-let-7b-5p,hsa-let-7b-5p,hsa-let-7b-5p
7,hsa-miR-106b-3p,hsa-miR-106b-3p,hsa-miR-106b-3p,hsa-miR-106b-3p,hsa-miR-106b-3p,hsa-miR-106b-3p,hsa-miR-106b-3p,hsa-miR-106b-3p,hsa-miR-106b-3p,hsa-miR-106b-3p,hsa-miR-106b-3p
8,hsa-miR-500a-5p,hsa-miR-500a-5p,hsa-miR-500a-5p,hsa-miR-500a-5p,hsa-miR-500a-5p,hsa-miR-500a-5p,hsa-miR-500a-5p,hsa-miR-500a-5p,hsa-miR-500a-5p,hsa-miR-500a-5p,hsa-miR-500a-5p
9,hsa-miR-486-3p,hsa-miR-486-3p,hsa-miR-486-3p,hsa-miR-486-3p,hsa-miR-486-3p,hsa-miR-486-3p,hsa-miR-486-3p,hsa-miR-486-3p,hsa-miR-486-3p,hsa-miR-486-3p,hsa-miR-486-3p


In [24]:
df_bd.to_csv('/home/ferambriz/inspecttionnn.tsv', sep = '\t', index = False)